# Important Data Structures (in Physics)

* Hash Tables
* DataFrames
* B-Trees
* K-d trees

## Hash Tables

In [1]:
hash('') == hash(0) == hash(0.0) == hash(False) == 0

True

## Pandas Data Frames and Series

### Series

The Pandas Series class is effectively a 1-dimentsional NumPy array with an associated index.

In [2]:
import pandas as pd

Series objects can be created from lists or NumPy arrays:

In [3]:
pd.Series([42, 43, 44], dtype='f8')

0    42.0
1    43.0
2    44.0
dtype: float64

An list of values can be supplied as an index. If no external index is given, an integer index is generated.

In [4]:
s = pd.Series([42, 43, 44], 
      index=["electron", 
             "proton", 
             "neutron"])

In [5]:
s

electron    42
proton      43
neutron     44
dtype: int64

Values can be accessed by their index:

In [6]:
s['electron']

42

In [7]:
# inclusive bounds
s['electron':'proton']

electron    42
proton      43
dtype: int64

In [12]:
# integer indexing still OK
s[1:]

proton     43
neutron    44
dtype: int64

Alternatively one can create a Series from a dict. In this case the dict's keys are used as index for the Series.

In [9]:
t = pd.Series({'electron': 6, 
              'neutron': 28, 
              'proton': 496, 
              'neutrino': 8128})

In [10]:
t

electron       6
neutrino    8128
neutron       28
proton       496
dtype: int64

Arithmetic with two Series objects works element-by-element. The elements are automatically aligned by their index. 
Indices that are only present in one Series, will get a NaN value in the resulting Series.

In [14]:
s + t

electron     48.0
neutrino      NaN
neutron      72.0
proton      539.0
dtype: float64

### Data Frames

Data Frames are reprenent a tabular (spreadsheet-like) data structure. They basically consist of columns of Series-like objects that have both a row- and column index.

In Chapter 07 (Analysis and Visualization) we have created a DataFrame by loading a CSV file with `pd.read_csv()` and Pandas can also read from HDF and Excel files. Data Frames can also be created from (nested) Python lists, NumPy arrays. Another way to create a DataFrame is to pass a dict of Series objects:

In [15]:
df = pd.DataFrame({'S': s, 'T': t})

In [16]:
df

,S,T
electron,42.0,6
neutrino,NaN,8128
neutron,44.0,28
proton,43.0,496


Data frames support NumPy like indexing by row: `df[ from : to : step ]`

In [20]:
df[::2]

,S,T
electron,42.0,6
neutron,44.0,28


Expand a dataframe with a new index and value in column 'S':

In [22]:
dg = df.append(pd.DataFrame({'S': [-8128]}, index=['antineutrino']))
dg

,S,T
electron,42.0,6.0
neutrino,NaN,8128.0
neutron,44.0,28.0
proton,43.0,496.0
antineutrino,-8128.0,NaN


In [23]:
dh = dg.drop('neutron')
dh

,S,T
electron,42.0,6.0
neutrino,NaN,8128.0
proton,43.0,496.0
antineutrino,-8128.0,NaN


Transpose DataFrame (works in NumPy arrays as well):

In [24]:
df.T

,electron,neutrino,neutron,proton
S,42.0,NaN,44.0,43.0
T,6.0,8128.0,28.0,496.0


Arithmetic can be applied to the whole data frame:

In [25]:
df < 42

,S,T
electron,False,True
neutrino,False,False
neutron,False,True
proton,False,False


In [26]:
# accessing a single column 
# will return a series
df['T']

electron       6
neutrino    8128
neutron       28
proton       496
Name: T, dtype: int64

In [28]:
# setting a name to a series
# or expression will add a 
# column to the frame
df['small'] = df['T'] < 100
df

,S,T,small
electron,42.0,6,True
neutrino,NaN,8128,False
neutron,44.0,28,True
proton,43.0,496,False


In [29]:
# deleting a column will
# remove it from the frame
del df['small']
df

,S,T
electron,42.0,6
neutrino,NaN,8128
neutron,44.0,28
proton,43.0,496


## B-Trees


In [22]:
from blist import sorteddict

In [23]:
b = sorteddict(first="Albert", 
               last="Einstein",
               birthday=[1879, 3, 14])
b

sorteddict({'birthday': [1879, 3, 14], 'first': 'Albert', 'last': 'Einstein'})

In [24]:
b['died'] = [1955, 4, 18]
b

sorteddict({'birthday': [1879, 3, 14], 'died': [1955, 4, 18], 'first': 'Albert', 'last': 'Einstein'})

In [25]:
list(b.keys())

['birthday', 'died', 'first', 'last']

## K-D Trees

In [26]:
class Node(object):
    
    def __init__(self, point, left=None, right=None):
        self.point = point
        self.left = left
        self.right = right
        
    def __repr__(self):
        isleaf = self.left is None and self.right is None
        s = repr(self.point)
        if not isleaf:
            s = "[" + s + ":"
        if self.left is not None:
            s += "\n  left = " + "\n  ".join(repr(self.left).split('\n'))
        if self.right is not None:
            s += "\n  right = " + "\n  ".join(repr(self.right).split('\n'))
        if not isleaf:
            s += "\n  ]"
        return s


def kdtree(points, depth=0):
    if len(points) == 0:
        return None
    k = len(points[0])
    a = depth % k
    points = sorted(points, key=lambda x: x[a])
    i = int(len(points) / 2)  # middle index, rounded down
    node_left = kdtree(points[:i], depth + 1)
    node_right = kdtree(points[i+1:], depth + 1)
    node = Node(points[i], node_left, node_right)
    return node

In [27]:
points = [(1, 2), (3, 2), 
          (5, 5), (2, 1), 
          (4, 3), (1, 5)]
root = kdtree(points)
print(root)

[(3, 2):
  left = [(1, 2):
    left = (2, 1)
    right = (1, 5)
    ]
  right = [(5, 5):
    left = (4, 3)
    ]
  ]


In [28]:
from scipy.spatial import KDTree
tree = KDTree(points)

In [29]:
tree.data

array([[1, 2],
       [3, 2],
       [5, 5],
       [2, 1],
       [4, 3],
       [1, 5]])

In [30]:
# query() defaults to only the closest point
dist, idx = tree.query([(4.5, 1.25)])

In [31]:
dist

array([ 1.67705098])

In [32]:
idx 

array([1])

In [33]:
# fancy index by idx to get the point
tree.data[idx]

array([[3, 2]])